# Visualisation de l'écosystème normatif des webservices OGC
## Définition des fonctions nécéssaires
### <u> edge_weight :</u> Définition du poid pour un noeud donné
* **<u>Entrées :</u>**
 * **ogc_nx :** L'instance nx.DiGraph() des noeuds que l'on souhaite analyser.
 * **standard :** Standard source (Déjà présent de base dans le fichier) à l'origine de la référence (départ de l'arête).
 * **ref :** Référence présente dans la source (destination de l'arête).
 * **OGCDict :** Dictionnaire de départ comportant toutes les normes et référence.
* <u>Sortie :</u>
 * **w :** Poid de la relation entre **standard :** et **ref :**

In [1]:
def edge_weight(ogc_nx,standard,ref,OGCDict):
    #Fréquence d'apparission de la référence dans le standard (Normal de rajouter +1 car il est tout de même cité dans la partie Référence normative)
    tf_ref = OGCDict[standard]["references"][ref] + 1

    #Nombre totale de standard de base.
    N = len(OGCDict.keys())
    
    #Nombre de fois que la référence est citée dans l'écosystème
    listedge = ogc_nx.edges()
    sf_ref = 1

    for edge in listedge:
        src, to = edge
        if to==ref:
            sf_ref=sf_ref+1

    isf_ref = math.log(len(OGCDict.keys())/sf_ref)
    #w = math.log(tf_ref * isf_ref)
    w = tf_ref * isf_ref
    
    return w
    #Ajout du paramètres de poids dans la relation
#    for edge in listedge:
#        if ogc_net.edges["from"] == src and ogc_net.edges["to"] == ref :
#            key, value = "value", w
#            listedge[listedge.index(edge)].update({key: value})

### <u>rgb_to_hex :</u> Simple convertion de couleurs RGB en Hexadécimal
* **<u>Entrées :</u>**
 * **rgb :** Tuple comportant les valeurs (r, g, b)
* **<u>Sortie :</u>**
 * Valeur hexadécimale brute

In [2]:
def rgb_to_hex(rgb):
  return '%02x%02x%02x' % rgb

## Ajout des noeuds à la base de données graphs

In [17]:
from pyvis.network import Network
import json
import networkx as nx
import math

#Mise en dico du fichier json
with open("test.json", 'r') as fp:
    InitOGCDict = json.load(fp)

#Visualisation du dico
ogc_net = Network(directed=True, notebook=True, cdn_resources='in_line')
ogc_nx = nx.DiGraph()

##Alimentation du réseau
##Initialisation des noeuds avec les normes de base
ogc_nx.add_nodes_from(InitOGCDict.keys())
print("Noeuds Principaux Ajouté !")
##Ajout du reste des noeuds qui ne font pas partie des normes de base
for key in InitOGCDict.keys():
    ogc_nx.add_nodes_from(InitOGCDict[key]["references"].keys())
print("Noeuds secondaire ajouté")
##Ajout des liens entre les différents 
for src in InitOGCDict.keys():
    for dest in InitOGCDict[src]["references"].keys():
        #Ajout du lien avec le calcul de son poid
        try:
            #print('Source : ' + src)
            #print('Référence : ' + dest)
            ogc_nx.add_edge(src,dest,value=edge_weight(ogc_nx,src,dest,InitOGCDict)) #Peut être voir pour mettre value à la place si pyvis comprend mieux
        except KeyError as e:
            print(e)
            #print("La relation " + src + " -> " + dest + " n'a pas pu être calculé")
        #Réglage de la taille des noeuds
        ogc_nx.nodes[src]['size']=len(InitOGCDict[src]['references'].keys())*3
        
#ogc_net.from_nx(ogc_nx)
#ogc_net.force_atlas_2based()
#ogc_net.prep_notebook()
#ogc_net.show("results/ogc_ws.html")

Noeuds Principaux Ajouté !
Noeuds secondaire ajouté


## Analyse par degré de centralité avec changement de couleurs des normes de base
* Couleur des noeuds :
 * Nuance de Rouge = référence
 * Vert/Blanc = norme de l'écosystème analysé

In [18]:
nx.degree_centrality(ogc_nx)
degree_centrality = nx.in_degree_centrality(ogc_nx)
max_key = max(degree_centrality, key=lambda key: degree_centrality[key])
max_degree = degree_centrality[max_key]
Verifdic=dict()
srclist=list()

for n in ogc_nx.nodes:
    if n in InitOGCDict.keys() :
        srclist.append(src)
        Verifdic["vert"] = srclist
        r=255*abs(((degree_centrality[n])/(max_degree)+0.1)-1)
        g=255*abs(((degree_centrality[n])/(max_degree)+0.1)-1)
        b=255
        rgb = (int(r),int(b),int(g))
        ogc_nx.nodes[n]['color']="#"+rgb_to_hex(rgb)
    else :
        srclist.append(src)
        Verifdic["rouge"] = srclist
        r=255
        g=255*abs((degree_centrality[n]/max_degree)-1)
        b=255*abs((degree_centrality[n]/max_degree)-1)
        rgb = (int(r),int(b),int(g))
        ogc_nx.nodes[n]['color']="#"+rgb_to_hex(rgb)

## Visualisation

In [19]:
ogc_net.from_nx(ogc_nx)
ogc_net.inherit_edge_colors(False)
#ogc_net.force_atlas_2based()
ogc_net.prep_notebook()
ogc_net.show("results/ogc_ws_centrality_dir.html")

results/ogc_ws_centrality_dir.html


### Résultat de voterank

In [20]:
nx.voterank(ogc_nx)

['OGC 20-004',
 'OGC 06-121r9',
 'OGC 19-087',
 'OGC 09-025r2',
 'OGC 18-062r2',
 'OGC 20-057',
 'OGC 15-001r4',
 'OGC 08-068r2',
 'OGC 09-083r4',
 'OGC 19-086r5',
 'OGC 22-026',
 'OGC 17-089r1',
 'OGC 06-042',
 'OGC 17-069r4',
 'OGC 05-005',
 'OGC 11-014r3',
 'OGC 19-072',
 'OGC 07-057r7',
 'OGC 14-065',
 'OGC 20-024']

## Récupération du poid des arêtes

In [21]:
print(ogc_nx.edges.data('value'))

[('OGC 05-005', 'IETF RFC 2045', 3.044522437723423), ('OGC 05-005', 'IETF RFC 2119', 6.089044875446846), ('OGC 05-005', 'SLD', 12.178089750893692), ('OGC 05-005', 'XML 1.0', 9.13356731317027), ('OGC 06-042', 'ISO 8601:2004', 21.31165706406396), ('OGC 06-042', 'ISO 19111', 24.356179501787384), ('OGC 06-042', 'ISO 19115', 18.26713462634054), ('OGC 06-042', 'IETF RFC 2045', 4.702750514326955), ('OGC 06-042', 'IETF RFC 2396', 18.26713462634054), ('OGC 06-042', 'IETF RFC 2616', 30.44522437723423), ('OGC 06-042', 'UCUM', 9.13356731317027), ('OGC 06-042', 'XML 1.0', 7.054125771490433), ('OGC 06-042', 'XML Schema', 12.178089750893692), ('OGC 06-121r9', 'CGI', 3.044522437723423), ('OGC 06-121r9', 'IETF RFC 2045', 3.8918202981106265), ('OGC 06-121r9', 'IETF RFC 2141', 3.044522437723423), ('OGC 06-121r9', 'IETF RFC 2396', 16.459626800144342), ('OGC 06-121r9', 'IETF RFC 2616', 21.1623773144713), ('OGC 06-121r9', 'IETF RFC 4646', 15.222612188617115), ('OGC 06-121r9', 'ISO/IEC Directives, Part 2', 9

In [11]:
ogc_nx['OGC 05-005']['HTML5']['value']

KeyError: 'HTML5'

In [22]:
G = ogc_nx
G['OGC 05-005']['IETF RFC 2045']['value']

3.044522437723423

In [23]:
for src in InitOGCDict.keys():
        rez=0
        nb_dep = 0
        for dest in ogc_nx.nodes():
            try :
                rez=rez+G[src][dest]['value']
                nb_dep = nb_dep + 1
            except KeyError :
                None
        print(str(src) + " à pour exposition : " + str(rez) + " pour " + str(nb_dep) + " N-1 dépendences.")

OGC 05-005 à pour exposition : 30.445224377234233 pour 4 N-1 dépendences.
OGC 06-042 à pour exposition : 145.71586354564798 pour 9 N-1 dépendences.
OGC 06-121r9 à pour exposition : 376.52794003169475 pour 26 N-1 dépendences.
OGC 07-057r7 à pour exposition : 253.23922976201138 pour 4 N-1 dépendences.
OGC 08-068r2 à pour exposition : 104.63299445846724 pour 9 N-1 dépendences.
OGC 09-025r2 à pour exposition : 443.2094596366952 pour 12 N-1 dépendences.
OGC 09-083r4 à pour exposition : 211.39153996325433 pour 6 N-1 dépendences.
OGC 11-014r3 à pour exposition : 45.36927706666602 pour 3 N-1 dépendences.
OGC 14-065 à pour exposition : 155.20384172311287 pour 6 N-1 dépendences.
OGC 15-001r4 à pour exposition : 68.90538905398331 pour 9 N-1 dépendences.
OGC 17-089r1 à pour exposition : 59.776088108832205 pour 6 N-1 dépendences.
OGC 19-072 à pour exposition : 138.44679099004753 pour 9 N-1 dépendences.
OGC 20-024 à pour exposition : 125.64122109311127 pour 8 N-1 dépendences.
OGC 19-086r5 à pour exp

In [15]:
print(ogc_nx.nodes)

['OGC 05-005', 'OGC 06-042', 'OGC 06-121r9', 'OGC 07-057r7', 'OGC 08-068r2', 'OGC 09-025r2', 'OGC 09-083r4', 'OGC 11-014r3', 'OGC 14-065', 'OGC 15-001r4', 'OGC 17-089r1', 'OGC 19-072', 'OGC 20-024', 'OGC 19-086r5', 'OGC 17-069r4', 'OGC 18-058r1', 'OGC 18-062r2', 'OGC 20-057', 'OGC 19-087', 'IETF RFC 2045', 'IETF RFC 2119', 'SLD', 'XML 1.0', 'ISO 8601:2004', 'ISO 19111', 'ISO 19115', 'IETF RFC 2396', 'IETF RFC 2616', 'UCUM', 'XML Schema', 'CGI', 'IETF RFC 2141', 'IETF RFC 4646', 'ISO/IEC Directives, Part 2', 'ISO 4217:2001', 'ISO 8601:2000', 'ISO 19119', 'ISO 19123', 'OGC 02-112', 'OGC 03-105r1', 'OGC 04-046r3', 'OGC 04-092r4', 'OGC 07-092r1', 'OGC 08-008r2', 'OGC 08-015', 'OGC 09-046r1', 'OGC 07-107r1', 'Namespaces In XML', 'SOAP', 'XML-binary Optimized Packaging', 'ISO 19105', 'OGC 06-121r3', 'WSDL 1.1', 'ISO 8601', 'OGC 05-007r7', 'OGC 07-011', 'OGC 07-067r5', 'OGC 08-053r2', 'OGC 08-059r3', 'ISO/TS 19103', 'ISO 19136', 'OGC 09-026r2', 'OGC 07-092r3', 'XML Path Language', 'ISO 19103'